In [37]:
# Globals to override
PROJCET_ID = ""
LOCATION_ID = "global"
AGENT_ID = "1f956f23-7fa3-42ea-8350-3d09f2714fc7"
LANGUAGE_CODE = 'en' 
JSON_KEY="~/df-insights-key.json"

In [38]:
key=! echo {JSON_KEY} 
%env GOOGLE_APPLICATION_CREDENTIALS={key[0]}
if PROJCET_ID == "" or PROJCET_ID is None:
    PROJCET_ID=!gcloud config get-value project
    if isinstance(PROJCET_ID,list): PROJCET_ID=PROJCET_ID[0] 

env: GOOGLE_APPLICATION_CREDENTIALS=/home/admin_edn_altostrat_com/df-insights-key.json


In [50]:
from IPython.display import Markdown as md

md(f"check out the conversation in insights - https://ccai.cloud.google.com/insights/projects/{PROJCET_ID}/conversations")


check out the conversation in insights - https://ccai.cloud.google.com/insights/projects/ccai-insights-test-edn/conversations

## Functions Code

In [41]:
from google.cloud import dialogflowcx_v3beta1 as dialogflow #cx
import uuid
from google.cloud.dialogflowcx_v3beta1.services.agents import AgentsClient
from google.cloud.dialogflowcx_v3beta1.services.sessions import SessionsClient
from google.cloud.dialogflowcx_v3beta1.types import session


In [43]:
def say(texts = ["Balance"],session_id = uuid.uuid4(),project_id=PROJCET_ID,location_id=LOCATION_ID,agent_id=AGENT_ID):
    agent = f"projects/{project_id}/locations/{location_id}/agents/{agent_id}"
    language_code = LANGUAGE_CODE

    responses = detect_intent_texts(agent, session_id, texts, language_code)
    for response in responses:
        print(f"Query text: {response.query_result.text}")
        response_messages = [
            " ".join(msg.text.text) for msg in response.query_result.response_messages
        ]
        print(f"Response text: {response_messages}\n")
    return session_id

In [44]:
def detect_intent_texts(agent, session_id, texts, language_code):
    """Returns the result of detect intent with texts as inputs.

    Using the same `session_id` between requests allows continuation
    of the conversation."""
    responses = []
    session_path = f"{agent}/sessions/{session_id}"
    #print(f"Session path: {session_path}\n")
    client_options = None
    agent_components = AgentsClient.parse_agent_path(agent)
    location_id = agent_components["location"]
    if location_id != "global":
        api_endpoint = f"{location_id}-dialogflow.googleapis.com:443"
        print(f"API Endpoint: {api_endpoint}\n")
        client_options = {"api_endpoint": api_endpoint}
    session_client = SessionsClient(client_options=client_options)

    texts = [texts] if isinstance(texts,str) else texts
    for text in texts:
        text_input = session.TextInput(text=text)
        query_input = session.QueryInput(text=text_input, language_code=language_code)
        request = session.DetectIntentRequest(
            session=session_path, query_input=query_input
        )
        response = session_client.detect_intent(request=request)
        responses.append(response)
    return responses



## Embeded Bot

- to use the dialog bot, you need to emable it in your project, navigate to **manage ->integrations & enable dialogflow messanger**  [docs](https://cloud.google.com/dialogflow/cx/docs/concept/integration/dialogflow-messenger)

In [60]:
html = f'''<script src="https://www.gstatic.com/dialogflow-console/fast/messenger-cx/bootstrap.js?v=1"></script>
<df-messenger
  df-cx="true"
  chat-title="Test Bot"
  agent-id="{AGENT_ID}"
  language-code="{LANGUAGE_CODE}"
></df-messenger>'''
from IPython.core.display import display, HTML
display(HTML(html))

## Dialog

In [55]:
session_id = say("Credit card balance amount")

Query text: Credit card balance amount
Response text: ['Sorry. Just the last 4 digits of the card please.', '']



In [56]:
session_id

UUID('352e7146-a230-4646-936a-179e0f091f60')

In [57]:
texts = ["again, I'm asking for a balance check"]
say(texts,session_id)

Query text: again, I'm asking for a balance check
Response text: ['Sorry. Just the last 4 digits of the card please.', '']



UUID('352e7146-a230-4646-936a-179e0f091f60')

In [ ]:
texts = ["for some reason this one does not work for me"]
say(texts,session_id)

Voice integration can be easily demomnstrated by enabling integration, getting a phone number and calling it from google voice TODO:Check if this can be done via APIs